In [5]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from utilities import *
import time
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
data = FashionMNISTLoader(resize=(128,128),batch_size=32)

# 定义 ViT 模型
model = ViT(image_size=128, patch_size=16, num_classes=10, dim=256, depth=2, heads=8, mlp_dim=1024, channels=1, dropout=0.1)

# 定义优化器函数和超参数
trainer_fn = torch.optim.RMSprop
hyperparams = {'lr': 0.1,'centered': True}
optimizer_name = 'RMSprop'
# 创建 Trainer 实例并训练模型
trainer = Trainer(trainer_fn, optimizer_name, hyperparams, data, model, feature_dim=10, num_epochs=4, cuda=True)
trainer.fit()
trainer.writer.close()

TypeError: __init__() got multiple values for argument 'feature_dim'

$$\mathbf{w} \leftarrow \mathbf{w} - \eta_t \mathbf{g}_t$$
# SGD（minibatch）
$$\mathbf{g}_t = \partial_{\mathbf{w}} \frac{1}{|\mathcal{B}_t|} \sum_{i \in \mathcal{B}_t} f(\mathbf{x}_{i}, \mathbf{w})$$

# Momentum
$$
\begin{aligned}
\mathbf{v}_t &\leftarrow \beta \mathbf{v}_{t-1} + \mathbf{g}_{t, t-1}, \\
\mathbf{w}_t &\leftarrow \mathbf{w}_{t-1} - \eta_t \mathbf{v}_t.
\end{aligned}
$$

# AdaGrad
$$\begin{aligned}
    \mathbf{s}_t & = \mathbf{s}_{t-1} + \mathbf{g}_t^2, \\
    \mathbf{w}_t & = \mathbf{w}_{t-1} - \frac{\eta}{\sqrt{\mathbf{s}_t + \epsilon}} \cdot \mathbf{g}_t.
\end{aligned}$$

# RMSProp
$$\begin{aligned}
    \mathbf{s}_t & \leftarrow \gamma \mathbf{s}_{t-1} + (1 - \gamma) \mathbf{g}_t^2, \\
    \mathbf{w}_t & \leftarrow \mathbf{w}_{t-1} - \frac{\eta}{\sqrt{\mathbf{s}_t + \epsilon}} \odot \mathbf{g}_t.
\end{aligned}$$

# Adam
$$\begin{aligned}
    \mathbf{v}_t & \leftarrow \beta_1 \mathbf{v}_{t-1} + (1 - \beta_1) \mathbf{g}_t, \\
    \mathbf{s}_t & \leftarrow \beta_2 \mathbf{s}_{t-1} + (1 - \beta_2) \mathbf{g}_t^2.
\end{aligned}$$
$$\hat{\mathbf{v}}_t = \frac{\mathbf{v}_t}{1 - \beta_1^t} \textrm{ and } \hat{\mathbf{s}}_t = \frac{\mathbf{s}_t}{1 - \beta_2^t}.$$
$$\mathbf{g}_t' = \frac{\eta \hat{\mathbf{v}}_t}{\sqrt{\hat{\mathbf{s}}_t} + \epsilon}.$$
$$\mathbf{x}_t \leftarrow \mathbf{x}_{t-1} - \mathbf{g}_t'.$$